In [18]:
import torch
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])

def corr2d(X,K):
  h,w=K.shape
  Y=torch.zeros(X.shape[0]-h+1,X.shape[1]-w+1)
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i,j]=(K*X[i:i+h,j:j+w]).sum()
      
  return Y


corr2d(X, K)
K[0][0],K[0,0],K[0:2][0:2]*K[0:2][0:2]

(tensor(0.),
 tensor(0.),
 tensor([[0., 1.],
         [4., 9.]]))

出的卷积层有时被称为特征映射（feature map），因为它可以被视为一个输入映射到下一层的空间维度的转换器。 在卷积神经网络中，对于某一层的任意元素 x ，其感受野（receptive field）是指在前向传播期间可能影响 x 计算的所有元素（来自所有先前层）。-

In [17]:
from torch import nn

X = torch.ones((6, 8))
X[:, 2:6] = 0
K=torch.tensor([[1.0,-1.0]])
Y = corr2d(X, K)  
lr=3e-2
conv2d=nn.Conv2d(1,1,(1,2),bias=False)
X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))
for epoch in range(10):
  
  Y_hat=conv2d(X)
  l=(Y_hat-Y)**2
  conv2d.zero_grad()
  l.sum().backward()
  conv2d.weight.data[:]-=lr*conv2d.weight.grad
  if (epoch+1)%2==0:
    print(f'loss{epoch}={l.sum()}')

print(conv2d.weight)

loss1=9.73404598236084
loss3=3.261800765991211
loss5=1.2143547534942627
loss7=0.4769849479198456
loss9=0.1919480264186859
Parameter containing:
tensor([[[[ 0.9460, -1.0360]]]], requires_grad=True)


# 6.3 padding and stride

在实践中，我们很少使用不一致的步幅或填充，

此外，使用奇数的核大小和填充大小也提供了书写上的便利。对于任何二维张量X，当满足： 1. 卷积核的大小是奇数； 2. 所有边的填充行数和列数相同； 3. 输出与输入具有相同高度和宽度 则可以得出：输出Y[i, j]是通过以输入X[i, j]为中心，与卷积核进行互相关计算得到的。

In [25]:
X=torch.randn(8,8)
print(X.shape,(1,1)+X.shape)

def show_result_shape(conv,X):
  X=X.reshape((1,1)+X.shape)
  Y=conv(X)
  return Y.reshape(Y.shape[2:])

#padding is set to ((h,w)-1)/2 to keep input,output same
conv2d=nn.Conv2d(1,1,(3,3),padding=1)
show_result_shape(conv2d,X).shape

conv2d=nn.Conv2d(1,1,(5,3),padding=(2,1))
show_result_shape(conv2d,X).shape


torch.Size([8, 8]) (1, 1, 8, 8)


torch.Size([8, 8])

# multi input channel

在最流行的神经网络架构中，随着神经网络层数的加深，我们常会增加输出通道的维数，通过减少空间分辨率以获得更大的通道深度。直观地说，我们可以将每个通道看作是对不同特征的响应。而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器

In [38]:
def corr2d_multi_in(X,K):
  #这里自动for第一维
  return sum(corr2d(x,k) for x,k in zip(X,K))

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

y=corr2d_multi_in(X, K)
print(y.shape,y)


torch.Size([2, 2]) tensor([[ 56.,  72.],
        [104., 120.]])


In [31]:
def corr2d_multi_in_out(X,K):
  return torch.stack([corr2d_multi_in(X,k) for k in K],0)

#stack 操作：指定维度，堆叠
print(K.shape)
K=torch.stack((K,K+1,K+2),0)
print(K.shape)
corr2d_multi_in_out(X,K)

torch.Size([2, 2, 2])
torch.Size([3, 2, 2, 2])


tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

# 1*1卷积层

可以将 1×1 卷积层看作是在每个像素位置应用的全连接层，以 ci 个输入值转换为 co 个输出值。

当执行 1×1 卷积运算时，上述函数相当于先前实现的互相关函数corr2d_multi_in_out。（why？

In [43]:
def corr2d_multi_in_out_1x1(X,K):
  c_i,h,w=X.shape
  c_o=K.shape[0]
  X=X.reshape(c_i,-1)
  K=K.reshape(c_o,-1)
  Y=torch.matmul(K,X)
  #@ for np, can't parellel
  Y=K@X
  return Y.reshape((c_o,h,w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6
print(float(torch.abs(Y1 - Y2).sum()))
print(Y1)

0.0
tensor([[[-2.5170,  2.4918, -0.3901],
         [ 2.2365, -0.9521,  1.6288],
         [ 0.9338, -1.9884,  1.4784]],

        [[-0.7450,  0.2705, -0.4235],
         [ 1.4650, -1.3130,  0.1807],
         [-0.3206, -0.5276,  0.0146]]])


疑问
1.for 多维，只循环最外维吗？还是说回根据函数参数（有点不可能

2.多维 矩阵乘法 还是不大懂唉，虽然好像先转成了2维

In [47]:
k=torch.arange(8)
k=k.reshape(2,2,2)
print(k.shape)
for x in k:
  print(x)



torch.Size([2, 2, 2])
tensor([[0, 1],
        [2, 3]])
tensor([[4, 5],
        [6, 7]])


# pooling
在处理多通道输入数据时，汇聚层在每个输入通道上单独运算，而不是像卷积层一样在通道上对输入进行汇总。 

In [48]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))

pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [53]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
print(X)
X=torch.cat((X,X+1),1)
print(X)

pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)



tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])
tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])


tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])